In [7]:
from google.colab import files
uploaded = files.upload()


Saving test.csv to test.csv


In [8]:
from google.colab import files
uploaded = files.upload()


Saving train.csv to train.csv


In [9]:
from google.colab import files
uploaded = files.upload()


Saving sample_submission.csv to sample_submission.csv


In [10]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

train.head()


,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [13]:
# shape (rows, columns)
train.shape

# column names
train.columns

# data types + null values
train.info()

# statistical summary
train.describe()

# check target distribution
train['y'].value_counts()


# check missing values
train.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         750000 non-null  int64 
 1   age        750000 non-null  int64 
 2   job        750000 non-null  object
 3   marital    750000 non-null  object
 4   education  750000 non-null  object
 5   default    750000 non-null  object
 6   balance    750000 non-null  int64 
 7   housing    750000 non-null  object
 8   loan       750000 non-null  object
 9   contact    750000 non-null  object
 10  day        750000 non-null  int64 
 11  month      750000 non-null  object
 12  duration   750000 non-null  int64 
 13  campaign   750000 non-null  int64 
 14  pdays      750000 non-null  int64 
 15  previous   750000 non-null  int64 
 16  poutcome   750000 non-null  object
 17  y          750000 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 103.0+ MB


,0
id,0
age,0
job,0
marital,0
education,0
default,0
balance,0
housing,0
loan,0
contact,0


In [14]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)


In [15]:
# separate target
y = train['y']
train = train.drop('y', axis=1)

# combine train + test for same encoding
full = pd.concat([train, test])

# find categorical columns
cat_cols = full.select_dtypes(include='object').columns

# apply label encoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cat_cols:
    full[col] = le.fit_transform(full[col])

# split back
train = full[:len(train)]
test = full[len(train):]

# add target back
train['y'] = y

train.head()

/tmp/ipython-input-3520924379.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['y'] = y


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,42,9,1,1,0,7,0,0,0,25,1,117,3,-1,0,3,0
1,38,1,1,1,0,514,0,0,2,18,6,185,1,-1,0,3,0
2,36,1,1,1,0,602,1,0,2,14,8,111,2,-1,0,3,0
3,27,8,2,1,0,34,1,0,2,28,8,10,2,-1,0,3,0
4,26,9,1,1,0,889,1,0,0,3,3,902,1,-1,0,3,1


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

# split features & target
X = train.drop('y', axis=1)
y = train['y']

# train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# create model
model = lgb.LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6
)

# train
model.fit(X_train, y_train)

# predict probabilities
val_pred = model.predict_proba(X_val)[:,1]

# calculate ROC-AUC
auc = roc_auc_score(y_val, val_pred)
print("Validation ROC-AUC:", auc)


[LightGBM] [Info] Number of positive: 72390, number of negative: 527610
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.070010 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120650 -> initscore=-1.986289
[LightGBM] [Info] Start training from score -1.986289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Validation ROC-AUC: 0.9663979653690893


In [17]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

X = train.drop('y', axis=1)
y = train['y']

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = []

for train_idx, val_idx in folds.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = lgb.LGBMClassifier(n_estimators=500, learning_rate=0.05)
    model.fit(X_train, y_train)

    pred = model.predict_proba(X_val)[:,1]
    auc = roc_auc_score(y_val, pred)
    auc_scores.append(auc)

print("Mean AUC:", np.mean(auc_scores))


[LightGBM] [Info] Number of positive: 72391, number of negative: 527609
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.146001 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120652 -> initscore=-1.986273
[LightGBM] [Info] Start training from score -1.986273
[LightGBM] [Info] Number of positive: 72391, number of negative: 527609
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.068920 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 16
[LightGBM] [Info

In [18]:
# ----- Feature Engineering -----

# ratios
train['balance_age_ratio'] = train['balance'] / (train['age'] + 1)
test['balance_age_ratio'] = test['balance'] / (test['age'] + 1)

train['campaign_day_ratio'] = train['campaign'] / (train['day'] + 1)
test['campaign_day_ratio'] = test['campaign'] / (test['day'] + 1)

# previous contact intensity
train['contact_intensity'] = train['previous'] + train['pdays']
test['contact_intensity'] = test['previous'] + test['pdays']

# customer engagement
train['total_contacts'] = train['campaign'] + train['previous']
test['total_contacts'] = test['campaign'] + test['previous']

# financial behaviour
train['loan_balance_ratio'] = train['balance'] / (train['campaign'] + 1)
test['loan_balance_ratio'] = test['balance'] / (test['campaign'] + 1)


/tmp/ipython-input-2650281052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['balance_age_ratio'] = train['balance'] / (train['age'] + 1)
/tmp/ipython-input-2650281052.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['balance_age_ratio'] = test['balance'] / (test['age'] + 1)
/tmp/ipython-input-2650281052.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [19]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# features & target
X = train.drop('y', axis=1)
y = train['y']

# k-fold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = []

for train_idx, val_idx in folds.split(X, y):

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # model
    model = lgb.LGBMClassifier(
        n_estimators=800,
        learning_rate=0.04,
        max_depth=7,
        num_leaves=50
    )

    # train
    model.fit(X_train, y_train)

    # probability prediction
    val_pred = model.predict_proba(X_val)[:,1]

    # ROC-AUC
    auc = roc_auc_score(y_val, val_pred)
    auc_scores.append(auc)

    print("Fold AUC:", auc)

print("Mean ROC-AUC:", np.mean(auc_scores))


[LightGBM] [Info] Number of positive: 72391, number of negative: 527609
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.289472 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2057
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120652 -> initscore=-1.986273
[LightGBM] [Info] Start training from score -1.986273
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [23]:
# Predict on test set
test_pred = model.predict_proba(test)[:, 1]  # or final_model if you trained on full data

# Create submission
submission = pd.DataFrame({
    'id': test['id'],
    'y': test_pred
})

# Save CSV
submission.to_csv("submission.csv", index=False)
print("submission.csv created ✅")


submission.csv created ✅


In [24]:
!ls


sample_data  sample_submission.csv  submission.csv  test.csv  train.csv


In [25]:
from google.colab import files
files.download("submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>